* enable 8 TPUs
* ResNet50 backbone
    * from v3: more accurate initial bias in the final layer
* mixup disabled
* use only TP samples
* cut 6sec around annotation.
* Stratified 5-fold and ensemble (just summing up probabilities)
    * from v3: clip-wise splitting instead of annotation-wise
* from v2: Grad-CAM visualization
* augmentation
    * gaussian noise ($\sigma=0.3$ while the image variance is 1)
    * random_brightness
    * specaugment
* v5: support TF2.4

In [ ]:
!pip install iterative-stratification

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from pathlib import Path
import io
import matplotlib.pyplot as plt
import soundfile as sf
import librosa
from kaggle_datasets import KaggleDatasets
from tqdm import tqdm
import pandas as pd
# from sklearn.model_selection import StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import seaborn as sns
from IPython.display import Audio
import cv2

tf.__version__

In [ ]:
cfg = {
    'parse_params': {
        'cut_time': 10,
    },
    'data_params': {
        'sample_time': 6, # assert 60 % sample_time == 0
        'spec_fmax': 24000.0,
        'spec_fmin': 40.0,
        'spec_mel': 224,
        'mel_power': 2,
        'img_shape': (224, 512)
    },
    'model_params': {
        'batchsize_per_tpu': 16,
        'iteration_per_epoch': 64,
        'epoch': 15,
        'arch': tf.keras.applications.ResNet50,
        'arch_preprocess': tf.keras.applications.resnet50.preprocess_input,
        'freeze_to': 0,  # Freeze to backbone.layers[:freeze_to]. If None, all layers in the backbone will be freezed.
        'loss': {
            'fn': tfa.losses.SigmoidFocalCrossEntropy,
            'params': {},
        },
        'optim': {
            'fn': tfa.optimizers.RectifiedAdam,
            'params': {'lr': 1e-3, 'total_steps': 15*64, 'warmup_proportion': 0.3, 'min_lr': 1e-6},
        },
        'mixup': False
    }
}

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
strategy = tf.distribute.TPUStrategy(tpu)
print("Number of devices: ", strategy.num_replicas_in_sync)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

TRAIN_TFREC = GCS_DS_PATH + "/tfrecords/train"
TEST_TFREC = GCS_DS_PATH + "/tfrecords/test"

In [ ]:
CUT = cfg['parse_params']['cut_time']
SR = 48000     # all wave's sample rate may be 48k

TIME = cfg['data_params']['sample_time']

FMAX = cfg['data_params']['spec_fmax']
FMIN = cfg['data_params']['spec_fmin']
N_MEL = cfg['data_params']['spec_mel']

HEIGHT, WIDTH = cfg['data_params']['img_shape']

CLASS_N = 24

# Explore the tfrecords, Create dataset

In [ ]:
raw_dataset = tf.data.TFRecordDataset([TRAIN_TFREC + '/00-148.tfrec'])
raw_dataset

## parse tfrecords

In [ ]:
feature_description = {
    'recording_id': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'audio_wav': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'label_info': tf.io.FixedLenFeature([], tf.string, default_value=''),
}
parse_dtype = {
    'audio_wav': tf.float32,
    'recording_id': tf.string,
    'species_id': tf.int32,
    'songtype_id': tf.int32,
    't_min': tf.float32,
    'f_min': tf.float32,
    't_max': tf.float32,
    'f_max':tf.float32,
    'is_tp': tf.int32
}

@tf.function
def _parse_function(example_proto):
    sample = tf.io.parse_single_example(example_proto, feature_description)
    wav, _ = tf.audio.decode_wav(sample['audio_wav'], desired_channels=1) # mono
    label_info = tf.strings.split(sample['label_info'], sep='"')[1]
    labels = tf.strings.split(label_info, sep=';')
    
    @tf.function
    def _cut_audio(label):
        items = tf.strings.split(label, sep=',')
        spid = tf.squeeze(tf.strings.to_number(items[0], tf.int32))
        soid = tf.squeeze(tf.strings.to_number(items[1], tf.int32))
        tmin = tf.squeeze(tf.strings.to_number(items[2]))
        fmin = tf.squeeze(tf.strings.to_number(items[3]))
        tmax = tf.squeeze(tf.strings.to_number(items[4]))
        fmax = tf.squeeze(tf.strings.to_number(items[5]))
        tp = tf.squeeze(tf.strings.to_number(items[6], tf.int32))

        tmax_s = tmax * tf.cast(SR, tf.float32)
        tmin_s = tmin * tf.cast(SR, tf.float32)
        cut_s = tf.cast(CUT * SR, tf.float32)
        all_s = tf.cast(60 * SR, tf.float32)
        tsize_s = tmax_s - tmin_s
        cut_min = tf.cast(
            tf.maximum(0.0, 
                tf.minimum(tmin_s - (cut_s - tsize_s) / 2,
                           tf.minimum(tmax_s + (cut_s - tsize_s) / 2, all_s) - cut_s)
            ), tf.int32
        )
        cut_max = cut_min + CUT * SR
        
        _sample = {
            'audio_wav': tf.reshape(wav[cut_min:cut_max], [CUT*SR]),
            'recording_id': sample['recording_id'],
            'species_id': spid,
            'songtype_id': soid,
            't_min': tmin - tf.cast(cut_min, tf.float32)/tf.cast(SR, tf.float32),
            'f_min': fmin,
            't_max': tmax - tf.cast(cut_min, tf.float32)/tf.cast(SR, tf.float32),
            'f_max': fmax,
            'is_tp': tp
        }
        return _sample
    
    samples = tf.map_fn(_cut_audio, labels, dtype=parse_dtype)
    return samples

parsed_dataset = raw_dataset.map(_parse_function).unbatch()

In [ ]:
@tf.function
def _cut_wav(x):
    # random cut in training
    cut_min = tf.random.uniform([], maxval=tf.minimum((CUT-TIME)*SR, tf.cast(x['t_max']*SR, tf.int32)), dtype=tf.int32)
    cut_max = cut_min + TIME * SR
    cutwave = tf.reshape(x['audio_wav'][cut_min:cut_max], [TIME*SR])
    y = {}
    y.update(x)
    y['audio_wav'] = cutwave
    y['t_min'] = tf.maximum(0.0, x['t_min'] - tf.cast(cut_min, tf.float32) / SR)
    y['t_max'] = tf.maximum(0.0, x['t_max'] - tf.cast(cut_min, tf.float32) / SR)
    return y
    
@tf.function
def _cut_wav_val(x):
    # center crop in validation
    cut_min = tf.minimum((CUT-TIME)*SR // 2, tf.cast((x['t_min'] + x['t_max']) / 2 * SR, tf.int32))
    cut_max = cut_min + TIME * SR
    cutwave = tf.reshape(x['audio_wav'][cut_min:cut_max], [TIME*SR])
    y = {}
    y.update(x)
    y['audio_wav'] = cutwave
    y['t_min'] = tf.maximum(0.0, x['t_min'] - tf.cast(cut_min, tf.float32) / SR)
    y['t_max'] = tf.maximum(0.0, x['t_max'] - tf.cast(cut_min, tf.float32) / SR)
    return y

In [ ]:
@tf.function
def _filtTP(x):
    return x['is_tp'] == 1

In [ ]:
def show_wav(sample, ax):
    wav = sample["audio_wav"].numpy()
    rate = SR
    ax.plot(np.arange(len(wav)) / rate, wav)
    ax.set_title(
        sample["recording_id"].numpy().decode()
        + ("/%d" % sample["species_id"])
        + ("TP" if sample["is_tp"] else "FP"))

    return Audio((wav * 2**15).astype(np.int16), rate=rate)

fig, ax = plt.subplots(figsize=(15, 3))
show_wav(next(iter(parsed_dataset)), ax)

## create mel-spectrogram

In [ ]:
@tf.function
def _wav_to_spec(x):
    mel_power = cfg['data_params']['mel_power']
    
    stfts = tf.signal.stft(x["audio_wav"], frame_length=2048, frame_step=512, fft_length=2048)
    spectrograms = tf.abs(stfts) ** mel_power

    # Warp the linear scale spectrograms into the mel-scale.
    num_spectrogram_bins = stfts.shape[-1]
    lower_edge_hertz, upper_edge_hertz, num_mel_bins = FMIN, FMAX, N_MEL
    
    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
      num_mel_bins, num_spectrogram_bins, SR, lower_edge_hertz,
      upper_edge_hertz)
    mel_spectrograms = tf.tensordot(
      spectrograms, linear_to_mel_weight_matrix, 1)
    mel_spectrograms.set_shape(spectrograms.shape[:-1].concatenate(
      linear_to_mel_weight_matrix.shape[-1:]))

    # Compute a stabilized log to get log-magnitude mel-scale spectrograms.
    log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)

    y = {
        'audio_spec': tf.transpose(log_mel_spectrograms), # (num_mel_bins, frames)
    }
    y.update(x)
    return y

spec_dataset = parsed_dataset.filter(_filtTP).map(_cut_wav).map(_wav_to_spec)

In [ ]:
plt.figure(figsize=(12,5))
for i, s in enumerate(spec_dataset.take(3)):
    plt.subplot(1,3,i+1)
    plt.imshow(s['audio_spec'])
plt.show()

In [ ]:
import librosa.display
import matplotlib.patches as patches

def show_spectrogram(sample, ax, showlabel=False):
    S_dB = sample["audio_spec"].numpy()
    img = librosa.display.specshow(S_dB, x_axis='time',
                             y_axis='mel', sr=SR,
                             fmax=FMAX, fmin=FMIN, ax=ax, cmap='magma')
    ax.set(title=f'Mel-frequency spectrogram of {sample["recording_id"].numpy().decode()}')
    sid, fmin, fmax, tmin, tmax, istp = (
            sample["species_id"], sample["f_min"], sample["f_max"], sample["t_min"], sample["t_max"], sample["is_tp"])
    ec = '#00ff00' if istp == 1 else '#0000ff'
    ax.add_patch(
        patches.Rectangle(xy=(tmin, fmin), width=tmax-tmin, height=fmax-fmin, ec=ec, fill=False)
    )

    if showlabel:
        ax.text(tmin, fmax, 
        f"{sid.numpy().item()} {'tp' if istp == 1 else 'fp'}",
        horizontalalignment='left', verticalalignment='bottom', color=ec, fontsize=16)

In [ ]:
fig, ax = plt.subplots(figsize=(15,3))
show_spectrogram(next(iter(spec_dataset)), ax, showlabel=True)

In [ ]:
# in validation, annotations will come to the center
fig, ax = plt.subplots(figsize=(15,3))
show_spectrogram(next(iter(parsed_dataset.filter(_filtTP).map(_cut_wav_val).map(_wav_to_spec))), ax, showlabel=True)

In [ ]:
for sample in spec_dataset.take(5):
    fig, ax = plt.subplots(figsize=(15,3))
    show_spectrogram(sample, ax, showlabel=True)

## create labels

In [ ]:
@tf.function
def _create_annot(x):
    targ = tf.one_hot(x["species_id"], CLASS_N, on_value=x["is_tp"], off_value=0)
    
    return {
        'input': x["audio_spec"],
        'target': tf.cast(targ, tf.float32)
    }

annot_dataset = spec_dataset.map(_create_annot)

## proprocessing and data augmentation

In training, I use

* gaussian noise
* random brightness
* specaugment

In [ ]:
@tf.function
def _preprocess_img(x, training=False):
    image = tf.expand_dims(x, axis=-1)
    image = tf.image.resize(image, [HEIGHT, WIDTH])
    image = tf.image.per_image_standardization(image)
    
    @tf.function
    def _specaugment(image):
        ERASE_TIME = 40
        ERASE_MEL = 20
        ERASE_TIME_N = 1
        ERASE_MEL_N = 1
        image = tf.expand_dims(image, axis=0)
        xoff = tf.random.uniform([ERASE_TIME_N], minval=ERASE_TIME//2, maxval=WIDTH-ERASE_TIME//2, dtype=tf.int32)
        xsize_half = tf.random.uniform([ERASE_TIME_N], minval=ERASE_TIME//4, maxval=ERASE_TIME//2, dtype=tf.int32)
        yoff = tf.random.uniform([ERASE_MEL_N], minval=ERASE_MEL//2, maxval=HEIGHT-ERASE_MEL//2, dtype=tf.int32)
        ysize_half = tf.random.uniform([ERASE_MEL_N], minval=ERASE_MEL//4, maxval=ERASE_MEL//2, dtype=tf.int32)
        for i in range(ERASE_TIME_N):
            image = tfa.image.cutout(image, [HEIGHT, xsize_half[i]*2], offset=[HEIGHT//2, xoff[i]])
        for i in range(ERASE_MEL_N):
            image = tfa.image.cutout(image, [ysize_half[i]*2, WIDTH], offset=[yoff[i], WIDTH//2])
        return tf.squeeze(image, axis=0)
    
    if training:
        # gaussian
        gau = tf.keras.layers.GaussianNoise(0.3)
        image = tf.cond(tf.random.uniform([]) < 0.5, lambda: gau(image, training=True), lambda: image)
        # brightness
        image = tf.image.random_brightness(image, 0.2)
        # specaugment
        image = tf.cond(tf.random.uniform([]) < 0.5, lambda: _specaugment(image), lambda: image, name='spec_cond')
        
    image = (image - tf.reduce_min(image)) / (tf.reduce_max(image) - tf.reduce_min(image)) * 255.0 # rescale to [0, 255]
    image = tf.image.grayscale_to_rgb(image)
    image = cfg['model_params']['arch_preprocess'](image)

    return image

@tf.function
def _preprocess(x):
    image = _preprocess_img(x['input'], True)
    return (image, x["target"])

@tf.function
def _preprocess_val(x):
    image = _preprocess_img(x['input'], False)
    return (image, x["target"])

@tf.function
def _preprocess_test(x):
    image = _preprocess_img(x['audio_spec'], False)
    return (image, x["recording_id"])

In [ ]:
plt.figure(figsize=(16, 4))
for i, (inp, targ) in enumerate(annot_dataset.map(_preprocess).take(6)):
    plt.subplot(2,3,i+1)
    plt.imshow(inp.numpy()[:,:,0])
    t = targ.numpy()
    if t.sum() == 0:
        plt.title(f'FP')
    else:
        plt.title(f'{t.nonzero()[0]}')
    plt.colorbar()
plt.show()

# Model

In [ ]:
def create_model():
    with strategy.scope():
        backbone = cfg['model_params']['arch'](include_top=False, weights='imagenet')
        
        if cfg['model_params']['freeze_to'] is None:
            for layer in backbone.layers:
                layer.trainable = False
        else:
            for layer in backbone.layers[:cfg['model_params']['freeze_to']]:
                layer.trainable = False
                
        # negative bias for stability (Section 4.1 in "Focal Loss" (https://arxiv.org/abs/1708.02002))
        prior = 1/CLASS_N
        pi = -np.log((1-prior) / prior) # sigmoid(pi) == prior

        head = tf.keras.Sequential([
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Dense(1024, activation='relu', kernel_initializer=tf.keras.initializers.he_normal()),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Dense(CLASS_N, bias_initializer=tf.keras.initializers.Constant(pi))
        ])
        model = tf.keras.Sequential([backbone, head])
    return model

model = create_model()
model.summary()

In [ ]:
@tf.function
def _mixup(inp, targ):
    indice = tf.range(len(inp))
    indice = tf.random.shuffle(indice)
    sinp = tf.gather(inp, indice, axis=0)
    starg = tf.gather(targ, indice, axis=0)
    
    alpha = 0.2
    t = tf.compat.v1.distributions.Beta(alpha, alpha).sample([len(inp)])
    tx = tf.reshape(t, [-1, 1, 1, 1])
    ty = tf.reshape(t, [-1, 1])
    x = inp * tx + sinp * (1-tx)
    y = targ * ty + starg * (1-ty)
#     y = tf.minimum(targ + starg, 1.0) # for multi-label???
    return x, y

In [ ]:
tfrecs = sorted(tf.io.gfile.glob(TRAIN_TFREC + '/*.tfrec'))
parsed_trainval = (tf.data.TFRecordDataset(tfrecs, num_parallel_reads=AUTOTUNE)
                    .map(_parse_function, num_parallel_calls=AUTOTUNE).unbatch()
                    .filter(_filtTP))

# Stratified 5-Fold

In [ ]:
df = pd.read_csv("/kaggle/input/rfcx-species-audio-detection/train_tp.csv")

table = df.groupby('recording_id')['species_id'].apply(
    lambda x: tf.scatter_nd(tf.expand_dims(x, 1), np.ones_like(x), shape=[CLASS_N]).numpy()
).reset_index()

skf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=0)
idx_splits = list(skf.split(table.recording_id, np.stack(table.species_id.to_numpy())))
splits = list(map(lambda xs: (table.recording_id[xs[0]].to_numpy(), table.recording_id[xs[1]].to_numpy()), idx_splits))

In [ ]:
_table = df.set_index('recording_id')
plt.hist([_table.loc[splits[0][0], 'species_id'], _table.loc[splits[0][1], 'species_id']],
         bins=CLASS_N,stacked=True)
plt.show()

In [ ]:
def create_recid_filter(recids):
    @tf.function
    def _filt(x):
        return tf.reduce_any(recids == x['recording_id'])
    return _filt

# Other setup

In [ ]:
def create_train_dataset(batchsize, train_recids):
    global parsed_trainval
    print("train split: %d" % len(train_recids))
    parsed_train = (parsed_trainval
                    .filter(create_recid_filter(train_recids)))

    dataset = (parsed_train.cache()
        .shuffle(len(train_recids))
        .repeat()
        .map(_cut_wav, num_parallel_calls=AUTOTUNE)
        .map(_wav_to_spec, num_parallel_calls=AUTOTUNE)
        .map(_create_annot, num_parallel_calls=AUTOTUNE)
        .map(_preprocess, num_parallel_calls=AUTOTUNE)
        .batch(batchsize))

    if cfg['model_params']['mixup']:
        dataset = (dataset.map(_mixup, num_parallel_calls=AUTOTUNE)
                    .prefetch(AUTOTUNE))
    else:
        dataset = dataset.prefetch(AUTOTUNE)
    return dataset

def create_val_dataset(batchsize, val_recids):
    global parsed_trainval
    print("val split: %d" % len(val_recids))
    parsed_val = (parsed_trainval
                  .filter(create_recid_filter(val_recids)))

    vdataset = (parsed_val
        .map(_cut_wav_val, num_parallel_calls=AUTOTUNE)
        .map(_wav_to_spec, num_parallel_calls=AUTOTUNE)
        .map(_create_annot, num_parallel_calls=AUTOTUNE)
        .map(_preprocess_val, num_parallel_calls=AUTOTUNE)
        .batch(8*strategy.num_replicas_in_sync)
        .cache())
    return vdataset

# Metrics

In [ ]:
# from https://www.kaggle.com/carlthome/l-lrap-metric-for-tf-keras
@tf.function
def _one_sample_positive_class_precisions(example):
    y_true, y_pred = example

    retrieved_classes = tf.argsort(y_pred, direction='DESCENDING')
    class_rankings = tf.argsort(retrieved_classes)
    retrieved_class_true = tf.gather(y_true, retrieved_classes)
    retrieved_cumulative_hits = tf.math.cumsum(tf.cast(retrieved_class_true, tf.float32))

    idx = tf.where(y_true)[:, 0]
    i = tf.boolean_mask(class_rankings, y_true)
    r = tf.gather(retrieved_cumulative_hits, i)
    c = 1 + tf.cast(i, tf.float32)
    precisions = r / c

    dense = tf.scatter_nd(idx[:, None], precisions, [y_pred.shape[0]])
    return dense

class LWLRAP(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name='lwlrap'):
        super().__init__(name=name)

        self._precisions = self.add_weight(
            name='per_class_cumulative_precision',
            shape=[num_classes],
            initializer='zeros',
        )

        self._counts = self.add_weight(
            name='per_class_cumulative_count',
            shape=[num_classes],
            initializer='zeros',
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        precisions = tf.map_fn(
            fn=_one_sample_positive_class_precisions,
            elems=(y_true, y_pred),
            dtype=(tf.float32),
        )

        increments = tf.cast(precisions > 0, tf.float32)
        total_increments = tf.reduce_sum(increments, axis=0)
        total_precisions = tf.reduce_sum(precisions, axis=0)

        self._precisions.assign_add(total_precisions)
        self._counts.assign_add(total_increments)        

    def result(self):
        per_class_lwlrap = self._precisions / tf.maximum(self._counts, 1.0)
        per_class_weight = self._counts / tf.reduce_sum(self._counts)
        overall_lwlrap = tf.reduce_sum(per_class_lwlrap * per_class_weight)
        return overall_lwlrap

    def reset_states(self):
        self._precisions.assign(self._precisions * 0)
        self._counts.assign(self._counts * 0)

# Testset and Inference function

In [ ]:
def _parse_function_test(example_proto):
    sample = tf.io.parse_single_example(example_proto, feature_description)
    wav, _ = tf.audio.decode_wav(sample['audio_wav'], desired_channels=1) # mono
    
    @tf.function
    def _cut_audio(i):
        _sample = {
            'audio_wav': tf.reshape(wav[i*SR*TIME:(i+1)*SR*TIME], [SR*TIME]),
            'recording_id': sample['recording_id']
        }
        return _sample

    return tf.map_fn(_cut_audio, tf.range(60//TIME), dtype={
        'audio_wav': tf.float32,
        'recording_id': tf.string
    })

def inference(model):
    tdataset = (tf.data.TFRecordDataset(tf.io.gfile.glob(TEST_TFREC + '/*.tfrec'), num_parallel_reads=AUTOTUNE)
        .map(_parse_function_test, num_parallel_calls=AUTOTUNE).unbatch()
        .map(_wav_to_spec, num_parallel_calls=AUTOTUNE)
        .map(_preprocess_test, num_parallel_calls=AUTOTUNE)
        .batch(128*(60//TIME)).prefetch(AUTOTUNE))
    
    rec_ids = []
    probs = []
    for inp, rec_id in tqdm(tdataset):
        with strategy.scope():
            pred = model.predict_on_batch(tf.reshape(inp, [-1, HEIGHT, WIDTH, 3]))
            prob = tf.sigmoid(pred)
            prob = tf.reduce_max(tf.reshape(prob, [-1, 60//TIME, CLASS_N]), axis=1)

        rec_id_stack = tf.reshape(rec_id, [-1, 60//TIME])
        for rec in rec_id.numpy():
            assert len(np.unique(rec)) == 1
        rec_ids.append(rec_id_stack.numpy()[:,0])
        probs.append(prob.numpy())
        
    crec_ids = np.concatenate(rec_ids)
    cprobs = np.concatenate(probs)
    
    sub = pd.DataFrame({
        'recording_id': list(map(lambda x: x.decode(), crec_ids.tolist())),
        **{f's{i}': cprobs[:,i] for i in range(CLASS_N)}
    })
    sub = sub.sort_values('recording_id')
    return sub

# Now start training!

In [ ]:
def plot_history(history, name):
    plt.figure(figsize=(8,3))
    plt.subplot(1,2,1)
    plt.plot(history.history["loss"])
    plt.plot(history.history["val_loss"])
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.title("loss")
    # plt.yscale('log')

    plt.subplot(1,2,2)
    plt.plot(history.history["lwlrap"])
    plt.plot(history.history["val_lwlrap"])
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.title("metric")

    plt.savefig(name)

In [ ]:
def train_and_inference(splits, split_id):
    batchsize = cfg['model_params']['batchsize_per_tpu'] * strategy.num_replicas_in_sync
    print("batchsize", batchsize)
    loss_fn = cfg['model_params']['loss']['fn'](from_logits=True, **cfg['model_params']['loss']['params'])

    idx_train_tf = tf.constant(splits[split_id][0])
    idx_val_tf = tf.constant(splits[split_id][1])

    dataset = create_train_dataset(batchsize, idx_train_tf)
    vdataset = create_val_dataset(batchsize, idx_val_tf)
    
    optimizer = cfg['model_params']['optim']['fn'](**cfg['model_params']['optim']['params'])
    model = create_model()
    with strategy.scope():
        model.compile(optimizer=optimizer, loss=loss_fn, metrics=[LWLRAP(CLASS_N)])

    cp_options = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")
    history = model.fit(dataset,
                        steps_per_epoch=cfg['model_params']['iteration_per_epoch'],
                        epochs=cfg['model_params']['epoch'],
                        validation_data=vdataset,
                        callbacks=[
                            tf.keras.callbacks.ModelCheckpoint(
                                filepath='model_best_%d.h5' % split_id,
                                save_weights_only=True,
                                monitor='val_lwlrap',
                                mode='max',
                                save_best_only=True,
                                options=cp_options,
                            ),
                        ])
    plot_history(history, 'history_%d.png' % split_id)
    
    ### inference ###
    model.load_weights('model_best_%d.h5' % split_id, options=cp_options)
    return inference(model), history

In [ ]:
# train and inference
# sub, _ = train_and_inference(splits, 0)

# N-fold ensemble
sub = sum(
    map(
        lambda i: train_and_inference(splits, i)[0].set_index('recording_id'),
        range(len(splits))
    )
).reset_index()

In [ ]:
sub.describe()

In [ ]:
sub.to_csv("submission.csv", index=False)

# Show Grad-CAM
Let's take a look at Grad-CAM and check if my model is trained properly

In [ ]:
model = create_model()

In [ ]:
# modified https://github.com/sicara/tf-explain/blob/8dff129ff7b1012dba2761a61e3c3e68e9ecbec2/tf_explain/core/grad_cam.py
"""MIT License

Copyright (c) 2019 SICARA

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE."""

def grad_cam(model, inputs, class_index):
    x = tf.keras.Input((None, None, 3))
    conv_y = model.get_layer(index=0)(x, training=False)
    y = model.get_layer(index=1)(conv_y, training=False)
    grad_model = tf.keras.Model(x, [conv_y, y])
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(inputs, training=False)
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)
    
    cams = []
    for grad, output in zip(grads, conv_outputs):
        weights = tf.reduce_mean(grad, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, output), axis=-1).numpy()
        cams.append(cam)
    return cams

In [ ]:
val_recids = splits[0][1]
parsed_val = (parsed_trainval
              .filter(create_recid_filter(val_recids)))

vdataset = (parsed_val
    .map(_cut_wav_val)
    .map(_wav_to_spec))

## before training

In [ ]:
fig, ax = plt.subplots(5, 2, figsize=(16,20))
for i, sample in enumerate(vdataset.take(5)):
    inpts = _preprocess_img(sample['audio_spec'], False)
    show_spectrogram(sample, ax[i,0], True)
    sid = sample['species_id']
    cams = grad_cam(model, tf.expand_dims(inpts, 0), sid)
    ax[i,1].set_title("grad-cam")
    ax[i,1].imshow(inpts[::-1,:,0], aspect='auto', interpolation='nearest', cmap='magma')
    ax[i,1].imshow(cv2.resize(cams[0][::-1], (WIDTH, HEIGHT)), cmap='magma', aspect='auto', interpolation='nearest', alpha=0.5)
plt.show()

## after training

In [ ]:
model.load_weights("./model_best_0.h5")
fig, ax = plt.subplots(5, 2, figsize=(16,20))
for i, sample in enumerate(vdataset.take(5)):
    inpts = _preprocess_img(sample['audio_spec'], False)
    show_spectrogram(sample, ax[i,0], True)
    sid = sample['species_id']
    cams = grad_cam(model, tf.expand_dims(inpts, 0), sid)
    ax[i,1].set_title("grad-cam")
    ax[i,1].imshow(inpts[::-1,:,0], aspect='auto', interpolation='nearest', cmap='magma')
    ax[i,1].imshow(cv2.resize(cams[0][::-1], (WIDTH, HEIGHT)), cmap='magma', aspect='auto', interpolation='nearest', alpha=0.5)
plt.show()

👍

In [ ]:
!mkdir gradcams

In [ ]:
for split_i in range(5):
    model.load_weights(f"./model_best_{split_i}.h5")
    fig, ax = plt.subplots(5, 2, figsize=(16,20))
    fig.suptitle(f"split {split_i}")
    for i, sample in enumerate(vdataset.take(5)):
        inpts = _preprocess_img(sample['audio_spec'], False)
        show_spectrogram(sample, ax[i,0], True)
        sid = sample['species_id']
        cams = grad_cam(model, tf.expand_dims(inpts, 0), sid)
        ax[i,1].set_title("grad-cam")
        ax[i,1].imshow(inpts[::-1,:,0], aspect='auto', interpolation='nearest', cmap='magma')
        ax[i,1].imshow(cv2.resize(cams[0][::-1], (WIDTH, HEIGHT)), cmap='magma', aspect='auto', interpolation='nearest', alpha=0.5)
    plt.savefig(f"./gradcams/split_{split_i}.png")